<a href="https://colab.research.google.com/github/xuxiufeng/Spam_ham_Classification/blob/main/Text%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unzip files

In [1]:
%%capture
!unzip enron1_test.zip
!unzip enron1_train.zip
!unzip enron4_test.zip
!unzip enron4_train.zip
!unzip enron1_test.zip
!unzip enron1_train.zip
!unzip hw1_test.zip
!unzip hw1_train.zip


## Stop Words

In [2]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 
              "you're", "you've","you'll", "you'd", 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his','himself', 'she', "she's", 'her', 
              'hers', 'herself', 'it', "it's", 'its', 'itself','they', 'them', 
              'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
              'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 
              'was', 'were', 'be','been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a','an', 'the', 'and', 'but', 'if', 
              'or', 'because', 'as', 'until', 'while', 'of', 'at','by', 'for', 
              'with', 'about', 'against', 'between', 'into', 'through', 'during',
              'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 
              'out', 'on','off', 'over', 'under', 'again', 'further', 'then', 'once', 
              'here', 'there', 'when','where', 'why', 'how', 'all', 'any', 'both', 
              'each', 'few', 'more', 'most', 'other','some', 'such', 'no', 'nor', 'not', 
              'only', 'own', 'same', 'so', 'than', 'too', 'very','s', 't', 'can', 'will', 
              'just', 'don', "don't", 'should', "should've", 'now', 'd','ll', 'm', 'o', 're', 
              've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 
              'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't",
              'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",
              'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',
              "won't", 'wouldn', "wouldn't"]

## Library

In [3]:
import copy
import glob
import os
import re
import random
import codecs
import numpy as np
from decimal import Decimal
from math import log10 as log
from collections import Counter
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

## Import Data

*  path: The path of folder. For example, 'content/enron1'
*  True_or_False_train_set: True means it is train set; otherwise, it is test set






In [16]:
def import_data_train_or_test(path, True_or_False_train_set):
    ham_raw_content = []
    spam_raw_content = []
    all_spam_ham_content = ""
    if True_or_False_train_set == True:
        path = path + '/train'
    else:
        path = path + '/test'
    path_ham = path + '/ham/'
    path_spam = path + '/spam/'
    spam_raw_contents_list = os.listdir(path_spam)
    ham_raw_contents_list = os.listdir(path_ham)
    for spam_raw_contents in spam_raw_contents_list:
      total_spam_path = path_spam + spam_raw_contents
      spam_raw_content.append(open(total_spam_path, "r", errors = 'ignore').read())
      all_spam_ham_content = all_spam_ham_content + " " + open(total_spam_path, "r", errors = 'ignore').read()
    for ham_raw_contents in ham_raw_contents_list:
      total_ham_path = path_ham + ham_raw_contents
      ham_raw_content.append(open(total_ham_path, "r", errors = 'ignore').read())
      all_spam_ham_content = all_spam_ham_content + " " + open(total_ham_path, "r", errors = 'ignore').read()
    # we find the size of the dataset and the number of instances with spam and number of instances with ham
    size_of_whole_dataset = len(ham_raw_contents_list) + len(spam_raw_contents_list)
    size_of_ham_dataset = len(ham_raw_contents_list)
    size_of_spam_dataset = len(spam_raw_contents_list)
    return spam_raw_content, ham_raw_content, all_spam_ham_content, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset

In [17]:
spam_raw_content, ham_raw_content, all_spam_ham_content, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset = import_data_train_or_test('/content/enron1', True)

In [18]:
size_of_whole_dataset

450

## Bag of Words model

In [13]:
def bag_of_words_model(dataset_name, True_or_False_train_set):
    """
    This function returns the bag of words for given dataset
    :param dataset_name: This is the dataset name
    :param True_or_False_train_set: In this if it is true then we have train data else test data
    :return: We return the bag for words representation for spam and ham files
    """
    spam_raw_content, ham_raw_content, all_spam_ham_content, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset = import_data_train_or_test(dataset_name, True_or_False_train_set)
    total_file_dictionary = {}
    total_file_data = re.findall("[a-zA-Z]+", all_spam_ham_content)
    text_frequency_in_all_folders = {}
    # at first we find all the words in the given dataset and find the frequencies in the whole dataset
    for each_word in total_file_data:
        each_word = each_word.lower()
        if each_word in total_file_dictionary:
            continue
        else:
            # I got the list of stem words from the nltk library
            if each_word not in stop_words:
                total_file_dictionary[each_word] = 0
        if each_word in text_frequency_in_all_folders:
            if each_word not in stop_words:
                text_frequency_in_all_folders[each_word] = text_frequency_in_all_folders[each_word] + 1
        else:
            if each_word not in stop_words:
                text_frequency_in_all_folders[each_word] = 1
    # In the following steps we find the words in the spam dataset and create the bag of words
    spam_email_bag_of_words = []
    spam_mail_in_all_documents = {}
    # The frequencies of each words are stored in the case of bag of words.
    for each_spam_mail in spam_raw_content:
        temp_dict = copy.deepcopy(total_file_dictionary)
        each_spam_mail1 = re.findall("[a-zA-Z]+", each_spam_mail)
        # Here we create the bag of words for each document and append it in a list
        for each_word in each_spam_mail1:
            each_word = each_word.lower()
            if each_word in temp_dict:
                temp_dict[each_word] = temp_dict[each_word] + 1
        # Here we store all the words in the spam dataset
        spam_mail_in_all_documents = Counter(spam_mail_in_all_documents) + Counter(temp_dict)
        spam_email_bag_of_words.append(temp_dict)
    # In the following steps we find all the ham words and add them in the bag of words
    ham_mail_in_all_documents = {}
    ham_email_bag_of_words = []
    for each_ham_mail in ham_raw_content:
        # Here we create the bag of words for each document and append it in a list
        temp_dict = copy.deepcopy(total_file_dictionary)
        each_ham_mail1 = re.findall("[a-zA-Z]+", each_ham_mail)
        for each_word in each_ham_mail1:
            each_word = each_word.lower()
            if each_word in temp_dict:
                temp_dict[each_word] = temp_dict[each_word] + 1
        # Here we store all the words in the ham dataset
        ham_mail_in_all_documents = Counter(ham_mail_in_all_documents) + Counter(temp_dict)
        ham_email_bag_of_words.append(temp_dict)
    return spam_email_bag_of_words, ham_email_bag_of_words, text_frequency_in_all_folders, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary

In [14]:
spam_email_bag_of_words, ham_email_bag_of_words, text_frequency_in_all_folders, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary = bag_of_words_model('/content/enron1', True)

In [ ]:
spam_email_bag_of_words

## Bernoulli model

In [ ]:
def convert_to_bernoulli_model(dataset_name, True_or_False_train_set):
    """
    This function returns the bernoulli model for given dataset
    :param dataset_name: This is the dataset name
    :param True_or_False_train_set: In this if it is true then we have train data else test data
    :return: We return the bernoulli model representation for spam and ham files
    """
    spam_raw_content, ham_raw_content, all_spam_ham_content, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset = import_data_train_or_test(
        dataset_name, True_or_False_train_set)
    total_file_dictionary = {}
    total_file_data = re.findall("[a-zA-Z]+", all_spam_ham_content)
    # at first we find all the words in the given dataset and find the occurrences in the whole dataset
    for each_word in total_file_data:
        # The words are converted to their lower case forms
        each_word = each_word.lower()
        if each_word in total_file_dictionary:
            continue
        else:
            if each_word not in stop_words:
                total_file_dictionary[each_word] = 0
    # In the following steps we find the words in the spam dataset and create the model
    spam_email_bernoulli_model = []
    spam_mail_in_all_documents = {}
    for each_spam_mail in spam_raw_content:
        # Here we create the bag of words for each document and append it in a list
        temp_dict = copy.deepcopy(total_file_dictionary)
        each_spam_mail1 = re.findall("[a-zA-Z]+", each_spam_mail)
        for each_word in each_spam_mail1:
            each_word = each_word.lower()
            if each_word in temp_dict:
                temp_dict[each_word] = 1
                # Here we store all the words in the ham dataset
                spam_mail_in_all_documents[each_word] = 1
        temp_list = list(temp_dict.values())
        spam_email_bernoulli_model.append(temp_dict)
    # In the following steps we find the words in the ham dataset and create the model
    ham_email_bernoulli_model = []
    ham_mail_in_all_documents = {}
    for each_ham_mail in ham_raw_content:
        # Here we create the bag of words for each document and append it in a list
        temp_dict = copy.deepcopy(total_file_dictionary)
        each_ham_mail1 = re.findall("[a-zA-Z]+", each_ham_mail)
        for each_word in each_ham_mail1:
            each_word = each_word.lower()
            if each_word in temp_dict:
                temp_dict[each_word] = 1
                ham_mail_in_all_documents[each_word] = 1
        ham_email_bernoulli_model.append(temp_dict)
    return spam_email_bernoulli_model, ham_email_bernoulli_model, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary

In [ ]:
spam_email_bernoulli_model, ham_email_bernoulli_model, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary= convert_to_bernoulli_model('/content/enron1', True)

## multinomial Naive Bayes

In [ ]:



def train_multinomial_NB(spam_email_bag_of_words, ham_email_bag_of_words, text_frequency_in_all_folders,
                         spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset,
                         size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary):
    """
    This is the main algorithm to train the multinomial Naive Bayes
    :param total_file_dictionary: This is the list containing all the words in the training examples
    :param spam_email_bag_of_words:  This is the list of all words in each spam document (1st value is for first document and so on)
    :param ham_email_bag_of_words: This is the list of all words in each ham document (1st value is for first document and so on)
    :param text_frequency_in_all_folders: This is the total text in all documents with their frequencies
    :param spam_mail_in_all_documents: This is the total text in all spam documents with their frequencies
    :param ham_mail_in_all_documents: This is the total text in all ham documents with their frequencies
    :param size_of_whole_dataset: This is total number of files in all dataset
    :param size_of_spam_dataset: This is total number of files in all spam dataset
    :param size_of_ham_dataset: This is total number of files in all ham dataset
    :return: prior and conditional probability for both spam and ham(all these values are in log)
    """
    no_of_docs = size_of_whole_dataset
    # We will first do it for the spam
    no_of_spam_docs = size_of_spam_dataset
    # We create the variables to store the values
    prior = {}
    conditional_probability = {}
    conditional_probability["spam"] = {}
    conditional_probability["ham"] = {}
    conditional_probability_of_non_occurring_word = {}
    conditional_probability_of_non_occurring_word["spam"] = {}
    conditional_probability_of_non_occurring_word["ham"] = {}
    value = Decimal(no_of_spam_docs / float(no_of_docs))
    # First we calculate the priors for the spam and ham dataset
    prior["spam"] = log(value)
    no_of_ham_docs = size_of_ham_dataset
    total_number_of_words_in_ham = sum(ham_mail_in_all_documents.values())
    prior["ham"] = log(no_of_ham_docs / float(no_of_docs))
    total_number_of_words_in_spam = sum(ham_mail_in_all_documents.values())
    # Now we calculate the values for the conditional probabilities
    for each_word in list(spam_mail_in_all_documents):
        conditional_probability["spam"][each_word] = log((spam_mail_in_all_documents[each_word] + 1) / (
            float(total_number_of_words_in_spam + len(text_frequency_in_all_folders))))

    # Now we will do the same procedure for ham docs
    for each_word in list(ham_mail_in_all_documents):
        conditional_probability["ham"][each_word] = log((ham_mail_in_all_documents[each_word] + 1) / (
            float(total_number_of_words_in_ham + len(text_frequency_in_all_folders))))
    # These are the values for the conditional probabilities whose words are not in the training dataset
    conditional_probability_of_non_occurring_word["ham"] = log(
        1 / (float(total_number_of_words_in_ham + len(text_frequency_in_all_folders))))
    conditional_probability_of_non_occurring_word["spam"] = log(
        1 / (float(total_number_of_words_in_spam + len(text_frequency_in_all_folders))))
    return prior, conditional_probability, conditional_probability_of_non_occurring_word

In [ ]:
def test_multinomial_naive_bayes(prior, conditional_probability, conditional_probability_of_non_occurring_word,
                                 an_email_bag_of_words_test):
    """
    :param conditional_probability_of_non_occurring_word: This is the conditional probability for each word in the testing set which is not in the training data
    :param prior: This is the prior for all classes
    :param conditional_probability:  This is the conditional probability for each word in vocabulary in spam and ham data
    :param an_email_bag_of_words_test: This is the given test instance we want to classify
    :return: the class of the given email
    """
    score = {}
    for each_class in list(prior):
        score[each_class] = prior[each_class]
        for each_word in list(an_email_bag_of_words_test):
            if an_email_bag_of_words_test[each_word] != 0:
                try:
                    score[each_class] += conditional_probability[each_class][each_word]
                # This is the case if the word was not in the train data and thus the laplace pruning gives this result
                except KeyError:
                    score[each_class] += conditional_probability_of_non_occurring_word[each_class]
    # Here we are taking spam as 1 and ham as -1
    if score["spam"] > score["ham"]:
        return 1
    else:
        return 0

In [ ]:
prior, conditional_probability, conditional_probability_of_non_occurring_word = train_multinomial_NB(spam_email_bag_of_words, ham_email_bag_of_words, text_frequency_in_all_folders,
                         spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset,
                         size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary)

In [ ]:
prior

{'spam': -0.5359412181195794, 'ham': -0.14942183071816256}

In [ ]:
def accuracy(true_y, predicted_y):
    """
    This function is used to calculate the accuracy
    :param true_y: These are the given values of class variable
    :param predicted_y: These are the predicted values of the class variable
    :return: the accuracy
    """
    accuracy_count = 0
    for each in range(len(true_y)):
        if true_y[each] == predicted_y[each]:
            accuracy_count = accuracy_count + 1
    return accuracy_count / float(len(true_y))


def precision(true_y, predicted_y):
    """
    This function is used to calculate the precision
    :param true_y: These are the given values of class variable
    :param predicted_y: These are the predicted values of the class variable
    :return: the precision
    """
    true_positives = 0
    false_positives = 0
    for each in range(len(true_y)):
        if true_y[each] == predicted_y[each] and predicted_y[each] == 1:
            true_positives += 1
        if true_y[each] != predicted_y[each] and predicted_y[each] == 1:
            false_positives += 1
    return true_positives / float(true_positives + false_positives)


def recall(true_y, predicted_y):
    """
    This function is used to calculate the recall
    :param true_y: These are the given values of class variable
    :param predicted_y: These are the predicted values of the class variable
    :return: the recall
    """
    true_positives = 0
    false_negetives = 0
    for each in range(len(true_y)):
        if true_y[each] == predicted_y[each] and predicted_y[each] == 1:
            true_positives += 1
        if true_y[each] != predicted_y[each] and predicted_y[each] == 0:
            false_negetives += 1
    return true_positives / float(true_positives + false_negetives)


def f1_score(recall, precision):
    """
    This function is used to calculate the f1_score
    :param recall: This is the value of recall
    :param precision: This is the value of precision
    :return: the f1_score
    """
    return (2 * recall * precision) / float(recall + precision)

In [ ]:
def evaluate_multinomial_NB(dataset_name):
    """
    This is the method used for evaluation of multinomial NB on a particular dataset
    :param dataset_name: This is the given dataset name
    :return: The method returns the accuracy, precision, recall and f1_score for the given dataset
    """
    # We first import training data for the training
    try:
        spam_email_bag_of_words, ham_email_bag_of_words, text_frequency_in_all_folders, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary = bag_of_words_model(
            dataset_name, True)
    except:
        print ("You have given wrong file name, please check and run again")
        exit(-1)
    prior, conditional_probability, conditional_probability_of_non_occurring_word = train_multinomial_NB(
        spam_email_bag_of_words, ham_email_bag_of_words, text_frequency_in_all_folders, spam_mail_in_all_documents,
        ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset,
        total_file_dictionary)
    # We now import the data for testing
    spam_email_bag_of_words, ham_email_bag_of_words, text_frequency_in_all_folders, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary = bag_of_words_model(
        dataset_name, False)
    # We calculate the evaluation metric
    # Here we first predict for the spam class and then the ham class
    spam_predict = []
    for each_document in spam_email_bag_of_words:
        spam_predict.append(test_multinomial_naive_bayes(prior, conditional_probability,
                                                                                  conditional_probability_of_non_occurring_word,
                                                                                  each_document))
    # We  are taking spam as 1
    spam_actual = [1] * len(spam_predict)
    ham_predict = []
    for each_document in ham_email_bag_of_words:
        ham_predict.append(test_multinomial_naive_bayes(prior, conditional_probability,
                                                                                 conditional_probability_of_non_occurring_word,
                                                                                 each_document))
    ham_actual = [0] * len(ham_predict)
    total_actual = spam_actual + ham_actual
    total_predict = spam_predict + ham_predict
    # Now we find the evaluation metrics for the method
    accur = accuracy(total_actual, total_predict)
    prec = precision(total_actual, total_predict)
    rec = recall(total_actual, total_predict)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1

In [ ]:
evaluate_multinomial_NB('/content/enron1')

(0.8114035087719298, 1.0, 0.4228187919463087, 0.5943396226415094)

## discrete Naive Bayes

In [ ]:


def discrete_naive_bayes_train(spam_email_bernoulli_model, ham_email_bernoulli_model,
                               spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset,
                               size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary):
    """
    
    :param total_file_dictionary: This is the total words list in the train dataset
    :param spam_email_bernoulli_model: This is the list of all the spam documents with bernoulli model
    :param ham_email_bernoulli_model: This is the list of all the ham documents with bernoulli model
    :param spam_mail_in_all_documents:This is the bernoulli model of all the spam documents
    :param ham_mail_in_all_documents:This is the bernoulli model of all the spam documents
    :param size_of_whole_dataset: This is total number of files in all dataset
    :param size_of_spam_dataset: This is total number of files in all spam dataset
    :param size_of_ham_dataset: This is total number of files in all ham dataset
    :return: estimate of prior and conditional probability
    """
    no_of_docs = size_of_whole_dataset
    no_of_spam_docs = size_of_spam_dataset
    no_of_ham_docs = size_of_ham_dataset
    prior = {}
    # We create variables to store the values
    conditional_probability = {}
    conditional_probability["spam"] = {}
    conditional_probability["ham"] = {}
    conditional_probability_of_non_occurring_word = {}
    conditional_probability_of_non_occurring_word["spam"] = {}
    conditional_probability_of_non_occurring_word["ham"] = {}
    # We calculate the prior of both the classes
    prior["spam"] = log(no_of_spam_docs / float(no_of_docs))
    prior["ham"] = log(no_of_ham_docs / float(no_of_docs))
    # We are doing 1-laplace smoothing and thus we add 1 in the numerator and 2 in denominator(since each word can
    # have two values o, 1 )
    for each_word in spam_mail_in_all_documents:
        conditional_probability["spam"][each_word] = log(
            1 + spam_mail_in_all_documents[each_word] / (float(no_of_spam_docs + 2)))

    for each_word in ham_mail_in_all_documents:
        conditional_probability["ham"][each_word] = log(
            1 + ham_mail_in_all_documents[each_word] / float(no_of_ham_docs + 2))
    # These are the probabilities for the word which are not in the training data and appear in the testing data
    conditional_probability_of_non_occurring_word["ham"] = log(1 / (float(no_of_ham_docs + 2)))
    conditional_probability_of_non_occurring_word["spam"] = log(1 / (float(no_of_spam_docs + 2)))
    return prior, conditional_probability, conditional_probability_of_non_occurring_word

In [ ]:
def discrete_naive_bayes_test(prior, conditional_probability, conditional_probability_of_non_occurring_word,
                              an_email_bag_of_words_test):
    """
    This is the function used to generate the output for the naive bayes algorithm
    :param prior: This is the prior generated from the naive bayes
    :param conditional_probability: This is the conditional probability generated from the naive bayes
    :param conditional_probability_of_non_occurring_word: This is the conditional probability of non occurring word generated from the naive bayes
    :param an_email_bag_of_words_test: This is the example on which we are going to test the algo
    :return: The class of the given instance
    """
    score = {}
    # In the following loop we find the words in the given documents for each class and find the posterior
    for each_class in list(prior):
        score[each_class] = prior[each_class]
        for each_word in list(an_email_bag_of_words_test):
            if an_email_bag_of_words_test[each_word] != 0:
                try:
                    score[each_class] += conditional_probability[each_class][each_word]
                # This is the case if the word was not in the train data and thus the laplace pruning gives this result
                except KeyError:
                    score[each_class] += conditional_probability_of_non_occurring_word[each_class]
    # Here we are taking spam as 1 and ham as -1
    if score["spam"] > score["ham"]:
        return 1
    else:
        return 0

In [ ]:
def evaluate_discrete_NB(dataset_name):
    """
    This is the method used for evaluation of multi-nomial NB on a particular dataset
    :param dataset_name: This is the given dataset name
    :return: We return the accuracy, precision, recall and f1_score for the given dataset
    """
    # We first import training data for the training
    try:
        spam_email_bag_of_words, ham_email_bag_of_words, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary = convert_to_bernoulli_model(
            dataset_name, True)
    except:
        print ("You have given wrong file name, please check and run again")
        exit(-1)
    prior, conditional_probability, conditional_probability_of_non_occurring_word = discrete_naive_bayes_train(
        spam_email_bag_of_words, ham_email_bag_of_words, spam_mail_in_all_documents,
        ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset,
        total_file_dictionary)
    # We now import the data for testing
    spam_email_bag_of_words, ham_email_bag_of_words, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary = convert_to_bernoulli_model(
        dataset_name, False)
    # We calculate the evaluation metric
    # Here we first predict for the spam class and then the ham class
    spam_predict = []
    for each_document in spam_email_bag_of_words:
        spam_predict.append(discrete_naive_bayes_test(prior, conditional_probability,
                                                                           conditional_probability_of_non_occurring_word,
                                                                           each_document))
    # We  are taking spam as 1
    spam_actual = [1] * len(spam_predict)
    ham_predict = []
    for each_document in ham_email_bag_of_words:
        ham_predict.append(discrete_naive_bayes_test(prior, conditional_probability,
                                                                          conditional_probability_of_non_occurring_word,
                                                                          each_document))
    ham_actual = [0] * len(ham_predict)
    total_actual = spam_actual + ham_actual
    total_predict = spam_predict + ham_predict
    # Now we predict the values for all the evaluation metrics
    accur = accuracy(total_actual, total_predict)
    prec = precision(total_actual, total_predict)
    rec = recall(total_actual, total_predict)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1
# evaluate_discrete_NB(dataset_name)

In [ ]:
evaluate_discrete_NB('/content/enron1')

(0.9342105263157895,
 0.8993288590604027,
 0.8993288590604027,
 0.8993288590604027)

## MCAP Logistic Regression

In [ ]:
def divide_into_validation_and_train(spam_mail_model, ham_mail_model):
    """
    This is the function used to divide the data into test and train data
    :param spam_mail_model: This is the representation(list) of each spam document in the given format
    :param ham_mail_model: This is the representation(list) of each ham document in the given format
    :return: the train and test set
    """
    # Here spam is 1 and ham is 0 (since we are using sigmoid)
    for each_dict in spam_mail_model:
        each_dict["this_is_the_class_of_the_document"] = 1
        each_dict["zero_weight"] = 1
    for each_dict in ham_mail_model:
        each_dict["this_is_the_class_of_the_document"] = 0
        each_dict["zero_weight"] = 1
    all_data = spam_mail_model + ham_mail_model
    # We are using this step to shuffle our data so that different data goes into training and testing everything
    random.shuffle(all_data)
    # 70 percent of the data is for traning and 30 percent of the data is for validation
    train_data = all_data[0: int(len(all_data) * .70)]
    validation_data = all_data[int(len(all_data) * .70): -1]
    return train_data, validation_data

In [ ]:
# train_data, validation_data = divide_into_validation_and_train(spam_email_bag_of_words, ham_email_bag_of_words)

In [ ]:
def get_output_for_class(weights, inputs):
    """
    This function is used to get the output for the given weights and inputs
    :param weights: These are the given weights
    :param inputs: These are the given inputs
    :return: We return the sum of product of individual values of weights and inputs
    """
    value = weights['zero_weight'] * 1
    for each in inputs:
        if each == 'this_is_the_class_of_the_document' or each == 'zero_weight':
            continue
        else:
            if each in weights and each in inputs:
                value = value + (weights[each] * inputs[each])
    return value

In [ ]:
def get_posterior(weights, inputs):
    """
    This function is used to get of the conditional log likelihood
    :param weights: These are the given weights
    :param inputs: These are the given inputs
    :return: We return the sum of product of individual values of weights and inputs
    """
    value = weights['zero_weight'] * 1
    for each in inputs:
        if each == 'this_is_the_class_of_the_document' or each == 'zero_weight':
            continue
        else:
            if each in weights and each in inputs:
                value = value + (weights[each] * inputs[each])
    return 1 / (float(1 + np.exp(-value)))

In [ ]:
def mcap_logistic_regression_train(train_data, total_file_dictionary, eta, lambda_parameter, number_of_iterations):
    """
    This function is used to train the log regression and find the optimum weights for the same
    :param number_of_iterations: These are the number of iteration we want to do for the algorithm
    :param train_data: This is the train data
    :param total_file_dictionary: This is the total list of words in the test data
    :param eta: This is the value of eta
    :param lambda_parameter: This is the value of lambda used for regularization
    :return: We return the optimized weights
    """
    # We are taking w_o outside the array
    weights = copy.deepcopy(total_file_dictionary)
    for each in weights:
        weights[each] = 0
    weights['zero_weight'] = 0
    # Now we update all the weights
    for each in range(number_of_iterations):
        for each_instance in train_data:
            posterior = get_posterior(weights, each_instance)
            sum_of_vals = 0
            for each_weight in weights:
                # Here I checked if the weight is not equal to 0 or not
                if each_instance[each_weight] != 0:
                    # This is the case when w_o is used
                    if each_weight == "zero_weight":
                        sum_of_vals = sum_of_vals + eta * (
                                each_instance["this_is_the_class_of_the_document"] - posterior)
                    else:
                        # This is the case when other w's are used
                        sum_of_vals = sum_of_vals + eta * (each_instance[each_weight] * (
                                each_instance["this_is_the_class_of_the_document"] - posterior))
                    weights[each_weight] = weights[each_weight] + sum_of_vals - eta * lambda_parameter * weights[
                        each_weight]
    return weights

In [ ]:
# weights = mcap_logistic_regression_train(train_data, total_file_dictionary, .1, 1, 10)

In [ ]:
def mcap_logistic_regression_test(test_example, weights):
    """
    This function is used to predict the output for the given test_example
    :param test_example: This is the given test example
    :param weights: These are the given weights
    :return: We return the class of the given instance
    """
    value = get_output_for_class(weights, test_example)
    # 0 is ham and 1 is spam
    if value < 0:
        return 0
    else:
        return 1

In [ ]:
def mcap_validation(train_data, validation_data, total_file_dictionary):
    """
    This function is for getting the best value for the parameter lambda
    :param train_data: This is the train data
    :param validation_data:  This is the validation data
    :param total_file_dictionary:  This is the list of all words.
    :return:
    """
    # Here I am doing the grid search for the lambda parameter
    eta = 0.01
    max_accuracy = 0
    best_lambda_value = 2
    # We take the range from 1 increasing 2 at a time
    for each_lambda_value in range(1, 8, 2):
        # We train the algo with the train data
        weights = mcap_logistic_regression_train(train_data, total_file_dictionary, eta, each_lambda_value, 50)
        correct_classification = 0
        # We test on the validation data
        for each_document in validation_data:
            output = mcap_logistic_regression_test(each_document, weights)
            if output == each_document["this_is_the_class_of_the_document"]:
                correct_classification = correct_classification + 1
        accuracy = correct_classification / float(len(validation_data))
        # Here we get the best lambda value
        if accuracy > max_accuracy:
            max_accuracy = accuracy
            best_lambda_value = each_lambda_value
    return best_lambda_value

In [ ]:
#mcap_validation(train_data, validation_data, total_file_dictionary)

In [ ]:
def evaluate_MCAP_bag_of_words(dataset_name):
    """
    This is the method used for evaluation of multinomial NB on a particular dataset
    :param dataset_name: This is the given dataset name
    :return: All the evaluation metrics
    """
    # We first import training data for the training
    try:
        spam_email_bag_of_words1, ham_email_bag_of_words1, text_frequency_in_all_folders, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary = bag_of_words_model(
            dataset_name, True)
    except:
        print ("You have given wrong file name, please check and run again")
        exit(-1)
    # Firstly we will divide our training data into training and validation data
    train_data, validation_data = divide_into_validation_and_train(spam_email_bag_of_words1,
                                                                                            ham_email_bag_of_words1)
    # Now we find the lambda value by using the grid search algorithm
    lambda_parameter = mcap_validation(train_data, validation_data, total_file_dictionary)
    # Here we merge the training data and the validation data again
    train_data = train_data + validation_data
    alpha_value = 0.01
    # In this step the algorithm learns the weights
    weights = mcap_logistic_regression_train(train_data, total_file_dictionary, alpha_value,
                                                                      lambda_parameter, 100)
    # We now import the data for testing
    spam_email_bag_of_words_test, ham_email_bag_of_words_test, text_frequency_in_all_folders_test, spam_mail_in_all_documents_test, ham_mail_in_all_documents_test, size_of_whole_dataset_test, size_of_spam_dataset_test, size_of_ham_dataset_test, total_file_dictionary_test = bag_of_words_model(
        dataset_name, False)
    spam_predict = []
    # In this step the algorithm predicts the output for a given dataset
    for each_document in spam_email_bag_of_words_test:
        spam_predict.append(mcap_logistic_regression_test(each_document, weights))
    # We  are taking spam as 1
    spam_actual = [1] * len(spam_predict)
    ham_predict = []
    for each_document in ham_email_bag_of_words_test:
        ham_predict.append(mcap_logistic_regression_test(each_document, weights))
    ham_actual = [0] * len(ham_predict)
    total_actual = spam_actual + ham_actual
    total_predict = spam_predict + ham_predict
    # Now we find the evaluation metrics for the method
    accur = accuracy(total_actual, total_predict)
    prec = precision(total_actual, total_predict)
    rec = recall(total_actual, total_predict)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1, lambda_parameter
    

In [ ]:
evaluate_MCAP_bag_of_words('/content/enron1')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


(0.9407894736842105,
 0.9485294117647058,
 0.8657718120805369,
 0.9052631578947368,
 3)

In [ ]:
def evaluate_MCAP_bernoulli_model(dataset_name):
    """
    This is the method used for evaluation of multinomial NB on a particular dataset
    :param dataset_name: This is the given dataset name
    :return: All the evaluation metrics
    """
    # We first import training data for the training
    try:
        spam_email_bernoulli_model1, ham_email_bernoulli_model1, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary = convert_to_bernoulli_model(
            dataset_name, True)
    except:
        print ("You have given wrong file name, please check and run again")
        exit(-1)
    # Firstly we will divide our training data into training and validation data
    train_data, validation_data = divide_into_validation_and_train(spam_email_bernoulli_model1,
                                                                                            ham_email_bernoulli_model1)
    # Now we find the lambda value by using the grid search algorithm
    lambda_parameter = mcap_validation(train_data, validation_data, total_file_dictionary)
    alpha_value = 0.01
    # Here we merge the training data and the validation data again
    train_data = train_data + validation_data
    # In this step the algorithm learns the weights
    weights = mcap_logistic_regression_train(train_data, total_file_dictionary, alpha_value,
                                                                      lambda_parameter, 100)
    # We now import the data for testing
    spam_email_bernoulli_model_test, ham_email_bernoulli_model_test, spam_mail_in_all_documents_test, ham_mail_in_all_documents_test, size_of_whole_dataset_test, size_of_spam_dataset_test, size_of_ham_dataset_test, total_file_dictionary_test = convert_to_bernoulli_model(
        dataset_name, False)
    spam_predict = []
    # In this step the algorithm predicts the output for a given dataset
    for each_document in spam_email_bernoulli_model_test:
        spam_predict.append(mcap_logistic_regression_test(each_document, weights))
    # We  are taking spam as 1
    spam_actual = [1] * len(spam_predict)
    ham_predict = []
    for each_document in ham_email_bernoulli_model_test:
        ham_predict.append(mcap_logistic_regression_test(each_document, weights))
    ham_actual = [0] * len(ham_predict)
    total_actual = spam_actual + ham_actual
    total_predict = spam_predict + ham_predict
    # Now we find the evaluation metrics for the method
    accur = accuracy(total_actual, total_predict)
    prec = precision(total_actual, total_predict)
    rec = recall(total_actual, total_predict)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1, lambda_parameter
# evaluate_MCAP_bag_of_words(dataset_name) #for bag of words
# evaluate_MCAP_bernoulli_model(dataset_name) # for bernoulli_model

In [ ]:
evaluate_MCAP_bernoulli_model('/content/enron1')

(0.8969298245614035,
 0.9811320754716981,
 0.697986577181208,
 0.8156862745098038,
 5)

## SGDClassifier

In [ ]:



def parameter_tuning(validation_x, validation_y):
    """
    This function tunes the parameters for the SGDClassifier and returns the classifier with optimized parameters
    :param validation_x: This is the validation attribute data
    :param validation_y: This is the class values for the given data
    :return: The tuned parameter
    """
    parameters_to_be_tuned = {'alpha': (0.01, 0.05),
                              'max_iter': (range(500, 3000, 1000)),
                              'learning_rate': ('optimal', 'invscaling', 'adaptive'),
                              'eta0': (0.3, 0.7),
                              'tol': (0.001, 0.005)
                              }
    SGDclassifier = SGDClassifier()
    gridSearch = GridSearchCV(SGDclassifier, parameters_to_be_tuned, cv=5)
    gridSearch.fit(validation_x, validation_y)
    return gridSearch


def train_SGD(train_x, train_y, classifier):
    """
    This is the function used to train the Stochastic Gradient Descent Algorithm
    :param train_x: This is train data
    :param train_y: This is the train labels/classes
    :param Classifier: This is the classifier after parameter tuning
    :return: This returns the trained classifier
    """
    return classifier.fit(train_x, train_y)


def test_SGD(trained_classifier, test_x, test_y):
    """
    This function is used to test the given classifier
    :param trained_classifier: This is the trained classifier we have got after training
    :param test_x: This is the test data
    :param test_y: These are the test classes
    :return: We return the accuracy of the given classifier
    """
    predicted_y = []
    for each_document in test_x:
        predicted_y.append(trained_classifier.predict(np.reshape(each_document, (1, -1))))
    return predicted_y, test_y


def convert_data_for_SGD_classifier(data, words_list):
    train_x = []
    train_y = []
    for each_document in data:
        train_x_for_this_document = []
        train_y.append(each_document["this_is_the_class_of_the_document"])
        for each_word in words_list:
            # We are using a try catch here since it may happen that the given word is not in the document
            try:
                train_x_for_this_document.append(each_document[each_word])
            except:
                # If the word is not in the test set then we just 0 as the input for the given word.
                train_x_for_this_document.append(0)
        train_x.append(train_x_for_this_document)
    return train_x, train_y


def get_data_from_given_model(spam_mail_model, ham_mail_model):
    """
    This is the function used to divide the data into test and train data
    :param spam_mail_model: This is the representation(list) of each spam document in the given format
    :param ham_mail_model: This is the representation(list) of each ham document in the given format
    :return: the train and test set
    """
    for each_dict in spam_mail_model:
        each_dict["this_is_the_class_of_the_document"] = 1
    for each_dict in ham_mail_model:
        each_dict["this_is_the_class_of_the_document"] = 0
    all_data = spam_mail_model + ham_mail_model
    # We are using this step to shuffle our data so that different data goes into training and testing everything
    return all_data


def divide_into_validation_and_train(spam_mail_model, ham_mail_model):
    """
    This is the function used to divide the data into test and train data
    :param spam_mail_model: This is the representation(list) of each spam document in the given format
    :param ham_mail_model: This is the representation(list) of each ham document in the given format
    :return: the train and test set
    """
    # Here spam is 1 and ham is 0 (since we are using sigmoid)
    for each_dict in spam_mail_model:
        each_dict["this_is_the_class_of_the_document"] = 1
    for each_dict in ham_mail_model:
        each_dict["this_is_the_class_of_the_document"] = 0
    all_data = spam_mail_model + ham_mail_model
    # We are using this step to shuffle our data so that different data goes into training and testing everything
    random.shuffle(all_data)
    train_data = all_data[0: int(len(all_data) * .70)]
    validation_data = all_data[int(len(all_data) * .70): -1]
    return train_data, validation_data

In [ ]:



def evaluate_SGD_bag_of_words(dataset_name):
    """
    This is the method used for evaluation of multinomial NB on a particular dataset
    :param dataset_name: This is the given dataset name
    :return: All the evaluation metrics
    """
    # We first import training data for the training
    try:
        spam_email_bag_of_words1, ham_email_bag_of_words1, text_frequency_in_all_folders, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary = bag_of_words_model(
            dataset_name, True)
        spam_email_bag_of_words_test, ham_email_bag_of_words_test, text_frequency_in_all_folders_test, spam_mail_in_all_documents_test, ham_mail_in_all_documents_test, size_of_whole_dataset_test, size_of_spam_dataset_test, size_of_ham_dataset_test, total_file_dictionary_test = bag_of_words_model(
            dataset_name, False)
    except:
        print( "You have given wrong file name, please check and run again")
        exit(-1)
    train_data, validation_data = divide_into_validation_and_train(spam_email_bag_of_words1,
                                                                                 ham_email_bag_of_words1)
    test_data = get_data_from_given_model(spam_email_bag_of_words_test, ham_email_bag_of_words_test)
    words_list = list(train_data[0])
    # we import the train, test and validation datasets
    train_x, train_y = convert_data_for_SGD_classifier(train_data, words_list)
    test_x, test_y = convert_data_for_SGD_classifier(test_data, words_list)
    valid_x, valid_y = convert_data_for_SGD_classifier(validation_data, words_list)
    # In this step we are getting the best parameters for the sklearn SGD classifier
    classifier_model = parameter_tuning(valid_x, valid_y)
    # In this step the classifier model is being trained on the training dataset
    trained_classifier_model = train_SGD(train_x, train_y, classifier_model)
    # In this step we find the output for the classifier.
    predicted_y, actual_y = test_SGD(trained_classifier_model, test_x, test_y)
    # Now calculate the evaluation metrics
    
    accur = accuracy(actual_y, predicted_y)
    prec = precision(actual_y, predicted_y)
    rec = recall(actual_y, predicted_y)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1


def evaluate_SGD_bernoulli_model(dataset_name):
    """
    This is the method used for evaluation of multinomial NB on a particular dataset
    :param dataset_name: This is the given dataset name
    :return: All the evaluation metrics
    """
    # We first import training data for the training
    try:
        spam_email_bernoulli_model1, ham_email_bernoulli_model1, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_whole_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary = convert_to_bernoulli_model(
            dataset_name, True)
        spam_email_bernoulli_model_test, ham_email_bernoulli_model_test, spam_mail_in_all_documents_test, ham_mail_in_all_documents_test, size_of_whole_dataset_test, size_of_spam_dataset_test, size_of_ham_dataset_test, total_file_dictionary_test = convert_to_bernoulli_model(
            dataset_name, False)
    except:
        print ("You have given wrong file name, please check and run again")
        exit(-1)
    train_data, validation_data = divide_into_validation_and_train(spam_email_bernoulli_model1,
                                                                                 ham_email_bernoulli_model1)
    test_data = get_data_from_given_model(spam_email_bernoulli_model_test, ham_email_bernoulli_model_test)
    words_list = list(train_data[0])
    # we import the train, test and validation datasets
    train_x, train_y = convert_data_for_SGD_classifier(train_data, words_list)
    test_x, test_y = convert_data_for_SGD_classifier(test_data, words_list)
    valid_x, valid_y = convert_data_for_SGD_classifier(validation_data, words_list)
    # In this step we are getting the best parameters for the sklearn SGD classifier
    classifier_model = parameter_tuning(valid_x, valid_y)
    # In this step the classifier model is being trained on the training dataset
    trained_classifier_model = train_SGD(train_x, train_y, classifier_model)
    # In this step we find the output for the classifier.
    predicted_y, actual_y = test_SGD(trained_classifier_model, test_x, test_y)
    # Now calculate the evaluation metrics
    accur = accuracy(actual_y, predicted_y)
    prec = precision(actual_y, predicted_y)
    rec = recall(actual_y, predicted_y)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1

# evaluate_SGD_bag_of_words(dataset_name) # for bow
# evaluate_SGD_bernoulli_model(dataset_name) # for bm

In [ ]:
evaluate_SGD_bag_of_words('/content/enron1')


(0.9912280701754386, 0.9738562091503268, 1.0, 0.9867549668874172)

In [ ]:
evaluate_SGD_bag_of_words('/content/enron1')

(0.9912280701754386, 0.9738562091503268, 1.0, 0.9867549668874172)